In [ ]:
%matplotlib notebook
%matplotlib inline

from scipy.special import binom
from scipy.optimize import minimize

import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider
import ipywidgets as widgets
from IPython.display import display

# Maximum Likelihood-Schätzer

Wir stellen für eine einfache Stichprobe $(x_1, \ldots, x_n)$, d.h. i.i.d gezogene $X_i$, die _Likelihood-Funktion_ bzw. _Likelihood_ in Abhängigkeit des unbekannten Parameters $\vartheta \in \Theta$ auf. Bei $\Theta$ handelt es sich um den kompletten Parameterraum:
$$L(\vartheta) = f(x_1,\ldots,x_n|\vartheta)\enspace,\enspace \vartheta \in \Theta$$

Damit definieren wir den __Maximum-Likelihood-Schätzer__ (__ML-Schätzer__) $\hat{\vartheta}_{ML}$ als:
$$ \hat{\vartheta} = \underset{\hat{\vartheta} \in \Theta}{\operatorname{argmax}}  L(\vartheta)$$
und für die Transformation mit Hilfe des streng monotonen Logarithmus zur __Log-Likelihoodfunktion__: 
$$ l(\vartheta) = \log L(\vartheta)$$
$$  \hat{\vartheta} = \underset{\hat{\vartheta} \in \Theta}{\operatorname{argmax}}  l(\vartheta)$$


__Beispiel__: 

Bernoulli-Versuch mit der Wahrscheinlichkeiten $p$ &rarr; $X_i \sim B(1,p)$. Damit ergibt sich die Likelihoodfunktion:
$$ \begin{eqnarray}
L(\vartheta) & = & f(x_1,\ldots,x_n|p)\enspace,\enspace p \in [0,1]\\
             & = & \prod_{i = 1}^n f(x_i|p) = \prod_{i = 1}^n p^x_i (1-p)^{1-x_i} \\
             & = & p^{\sum_{i=1}^n x_i} (1-p)^{\sum_{i=1}^n {1-x_i}} \\
             & = & p^{n\overline{x}} (1-p)^{n(1-\overline{x})}\end{eqnarray}$$
             
Bestimmen wir den Maximum-Likelihood-Schätzer für die Wahrscheinlichkeit $p$:
$$ \begin{eqnarray}
L(\vartheta) & = & p^{n\overline{x}} (1-p)^{n(1-\overline{x})}\\
l(\vartheta) & = & n\overline{x} \cdot \ln p + n(1-\overline{x})\cdot \ln(1-p)\\
\text{1. Abl.} = 0: \enspace \frac{\mathrm{d}l(\vartheta)}{\mathrm{d} p} & = & n\overline{x} \frac{1}{p} - n(1-\overline{x})\frac {1}{1-p} = 0 \\
\hat{p}_{ML} & = & \overline{x} \\
\text{2. Abl.} < 0: \enspace \frac{\mathrm{d}^2 l(\vartheta)}{\mathrm{d} p^2} & = & - n\overline{x} \frac{1}{p^2} - n(1-\overline{x})\frac {1}{(1-p)^2} < 0 
\end{eqnarray}$$
Damit ist die relative Häufigkeit zugleich der ML-Schätzer für $p$:
$$\hat{p}_{ML} = \frac{1}{n}\sum_{i=1}^n x_i = \overline{x}$$
Wir haben in einer Stichprobe von $n$ Werten $k$ Realisierungen erhalten. Wie sieht die Likelihoodfunktion aus: 

In [ ]:
def L(p,n,x):
    return p**x * (1-p)**(n-x)

def argmax_L(n,x):
    return float(x)/n

def binomial_likelihood(n,k):
    fig, ax = plt.subplots(1, 1)
    x = np.linspace(0, 1, 100)
    ax.set_xlabel("p")
    ax.plot(x, L(x, n, k), 'r-', lw=2, alpha=0.9, label=f'L({k} von {n}| p)')
    ax.axvline(x = argmax_L(n,k), label=f'Max. Likelihood-Est. {k}/{n}')
    ax.legend(loc='best', frameon=False)
    ax.grid(True)
    plt.show()

# Define sliders for n and k:
n_slider = IntSlider(min=1, max=50, value=10, continuous_update=False)
k_slider = IntSlider(min=0, max=10, value=0, continuous_update=False)

def update_k_range(*args):
    k_slider.max = n_slider.value
n_slider.observe(update_k_range, 'value')

# Define widget for the Likelihood function:
binomial_likelihood_widget = interactive(binomial_likelihood,
                                      n = n_slider,
                                      k = k_slider)
display(binomial_likelihood_widget)

## Beispiel mit Numerischer Optimierung

### Schätzung statistischer Kenngrößen

Beispiel entnommen aus " _Methoden der statistischen Inferenz - Likelihood und Bayes_ ", Leonhard Held, Spektrum Akademischer Verlag Heidelberg 2008.

Testverfahren dienen der Früherkennund von Krankheiten, entsprechend sollten sich die Tests durch eine hohe:
* __Sensitivität__ = P(Test ist positiv | Proband ist krank)
* __Spezifität__ = P(Test ist negativ | Proband ist gesund)

auszeichnen. Insbesondere ist es wünschenswert eine niedrige

* __Falsch-Negativ-Rage__ = P(Test ist negativ | Proband ist krank)

zu erreichen.

### Beispiel - Darmkrebs-Screening

Screening-Untersuchung für Darmkrebs im Rahmen einer australischen Studie an 38000 Patienten. An 6 aufeinanderfolgenden Tagen wurde je ein Test durchgeführt. Bei den 3000 Patienten, bei denen mindestens einer dieser Tests positiv war, wurde mit einer Darmspiegelung das Testergebnis überprüft. Es ergab sich die folgende Häufigkeitsverteilung $Z_k$ für die positiven Testergebnisse bei 196 Krebskranken:

| Anzahl der k positiven Tests  | 0  | 1  | 2  | 3  | 4 | 5 | 6 |
|:-:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| Häufigkeit $Z_k$  | NA  | 37  | 22  | 25  | 29 | 34 | 49 |

__Aufgrund des gewählten Studiendesigns fehlt die Angabe der Kranken, für die keiner der 6 Tests angeschlagen hat!__
&rarr; Wir müssen die __Falsch-Negativ-Rate__ schätzen!

$p$ ist die Wahrscheinlichkeit eines positiven Testergebnisses für einen Darmkrebskranken und $X_i$ die Anzahl an positiven Testergebnissen für einen Darmkrebskranken $i$. $p$ soll in der Population nicht variieren &rarr; $X_i$ ist binomial verteilt:

$$X_i \sim B(N,p) \enspace \text{mit} \enspace N = 6$$

wobei aber $X_0$ nicht beobachtet wurde &rarr; _gestutzte Binomialverteilung_ :

$$P(X_i = k | X_i > 0) = \frac{P(X_i=k)}{P(X_i>0)}, \enspace k=1,\ldots,6$$

was auf die Log-Likelihoodfunktion führt:

$$ l(p) = \sum_{k=1}^N Z_k (k \ln(p) + (N-k)\ln(1-p)) - n \ln(1-(1-p)^N) $$

$Z_k$ ist die Anzahl der Patienten mit $k$ positiven Testergebnissen und $n = \sum_{k=1}^N Z_k = 196$ die Gesamtzahl der Patienten mit mindestens einem positiven Testergebnis.



In [ ]:
Z = np.array([37,22,25,29,34,49])

def l(p, Z):
    n = Z.sum()
    N = len(Z)
    vec = np.array(range(6)) + 1
    return sum(Z * (vec * np.log(p) + (N - vec) * np.log(1 - p))) - n * np.log(1 - (1- p)**N)

In [ ]:
def plot_l(p = None):
    fig, ax = plt.subplots(1, 1)
    x = np.linspace(0.01, 0.99, 100)
    ax.set_xlabel("p")
    ax.plot(x, [l(p,Z) for p in x], 'r-', lw=2, alpha=0.9, label=f'l(p)')
    if p is not None:
        ax.axvline(x = p, label=f'Max. Likelihood-Est. p')
    ax.legend(loc='best', frameon=False)
    ax.grid(True)
    plt.show()
    
plot_l()

Numerisches Suchen des Maximum-Likelihood Schätzers, indem wir das Minimum von $-l(p)$ bestimmen:

In [ ]:
def neg_l(p):
    print(f'p: {p[0]}')
    return - 1*l(p,Z)

#result = minimize(neg_l,[0.1,0.9],method="L-BFGS-B", options={'ftol': 1e-6, 'disp': True})
result = minimize(neg_l, 0.5, bounds=[(0.01,0.99)], method="L-BFGS-B", options={'ftol': 1e-7, 'disp': True})
p = round(result.x[0],4)
print(f'ML-Schätzer für p: {p}')

In [ ]:
plot_l(p)

Damit gitl für die Falsch-Negativ-Rate:
$$ \gamma = P(X_i = 0) \to \hat{\gamma} = (1 - \hat{p}_{ML})^N$$
und damit für $Z_0$:
$$ \hat{Z}_0 = n \cdot \frac{\gamma}{1 - \gamma}$$

In [ ]:
gamma = (1-p)**len(Z)
print(f'Gamma (FNR):{round(gamma,4)}')
Z_0 = Z.sum() * gamma / (1 - gamma)
print(f'Z_0:{round(Z_0,2)}')

### Berechnung mit dem EM-Algorithmus

Der __EM-Algorithmus (Expectation-Maximization-Algorithm)__ kann iterativ ebenfalls numerisch einen Maximum-Likelihood-Schätzer bestimmen.

Für unser Beispiel heißt das:

Wären alle Daten für $Z_i$ bekannt, also auch für $Z_0$, ließe sich der Schätzer $\hat{p}$ relativ einfach bestimmen:
$$ \hat{p} = \frac{\sum_{k=0}^{6}k\cdot Z_k}{6\cdot \sum_{k=0}^{6} Z_k}$$
$Z_0$ ist aber nicht bekannt. Für ein festes $p$ könnte man aber $Z_0$ berechnen, wie eben gerade als:
$$ \hat{Z}_0 = n \cdot \frac{\gamma}{1 - \gamma}$$
mit $n$ und $\gamma=(1-p)^N$. Mit Hilfe des EM-Algorithmus iteriert man über beide Ausdrücke und bestimmt die beiden Schätzer $\hat{Z}_0$ und $\hat{p}$, die man in der nächsten Iteration für $Z_0$ und $p$ einsetzt:

In [ ]:
def EM(verbose = False):
    p = 0.5
    Z_0 = 10
    Z = np.array([Z_0,37.0,22.0,25.0,29.0,34.0,49.0])
    n = Z[1:].sum()
    delta = 1
    k = np.array(range(7))
    if verbose:
            counter = 0
            print(f'{counter}. p: {p}, Z_0: {Z_0}')
    while (delta > 1e-07):
        old_Z_0 = Z[0]
        p = np.sum(Z * k) / (6 * Z.sum())
        gamma = (1 - p)**6
        Z_0 = n * gamma / (1 - gamma)
        delta = abs(old_Z_0 - Z_0)
        Z[0] = Z_0
        if verbose:
            counter += 1
            print(f'{counter}. p: {p}, Z_0: {Z_0}, Gamma: {gamma}')
    return (p, Z_0, gamma)

em_p, em_Z_0, em_gamma = EM(True)
print(f'p: {round(em_p,4)} und Z_0: {round(em_Z_0,2)} und Gamma: {round(em_gamma,4)}')